Google News Scrapper con Analisis de sentimiento 

In [6]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
import openai
from openai import OpenAI
from newspaper import Article
from newspaper.article import ArticleException

In [7]:

client = OpenAI(api_key="tuclaveOpenAI")

# Maxima cantidad de dias desde la publicacion del articulo hasta el dia de la fecha 
cantDias = 25

#Base de datos de articulos
df = pd.DataFrame(columns=["Query", "Titulo","Fecha","Url","Texto","Sentimiento"])


#Categorias de busquedo y las queries asociadas a la misma
queries = {
    "Politica": ["Milei","Caputo"],
    "Economia": ["Comercio","Finanzas","Pyme"],
    "Actual": ["RIGI"]
    }


'''
Diccionario para url's de articulos segun su categoria. Respeta del orden de busqueda, i.e. el primer articulo 
scrapeado para coyuntura sera el primer url de la clave Coyuntura
'''
articulos_links = {key: [] for key in queries}

articulos_titulo = {key: [] for key in queries}


def extract_article_data(url):
    try:
        articloTemp = Article(url)
        articloTemp.download()
        articloTemp.parse()
        
        titulo = articloTemp.title
        fecha = articloTemp.publish_date.strftime("%d-%m-%Y") if articloTemp.publish_date else None
        texto = articloTemp.text
        fuente = articloTemp.source_url if articloTemp.publish_date else None

        return titulo, fecha, texto, fuente
    except ArticleException:
        return None, None, None, None


def process_article(driver, article_xpath,query):
    #Buscamos el articulo por xpath por hasta 10seg
    try:
        article_title = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, article_xpath))
        )
        #Guardamos el Url del articulo
        #NOTA: Este es un url que se redirige a el url de verddad
        url_sin_redirigir = article_title.get_attribute("href")

        #Vamos a la pagina a lo que nos redirige google news y guardamos el url final
        driver.get(url_sin_redirigir)
        time.sleep(1)
        url_final = driver.current_url

        #Retornamos al driver a la pagina con el query actual
        driver.get(url_google_news)

        #Guardamos el url del articulo en bajo su respectiva clave
        articulos_links[key].append(url_final)

        #Usando Newspaper3k extraemos la info del articulo
        #OBS; A veces falla la fecha, el titulo y el texto suelen extraerse bien
        titulo, fecha, texto, fuente = extract_article_data(url_final)

        articulos_titulo[key].append(titulo)

        #Con el fin de almacen la fuente sin caracteres de mas removemos algunas palabras innecesarias
        if fuente is not None:
            fuente = fuente.replace('https://', '')
            fuente = fuente.replace('www.', '')
            fuente = fuente.replace('.com', '')
            fuente = fuente.replace('.ar', '')
            fuente = fuente.replace('.net', '')
            fuente = fuente.replace('.org', '')

        if texto is not None and texto != "":

            #resumen = resumenArticulo(texto)
            completion = client.chat.completions.create(
                    #Aca estamos usando el modelo mas barato, sale $0.001 USD y tenes $5 USD menseuales gratuitos
                    model="gpt-3.5-turbo",
                    messages=[
                    {"role": "user", "content": f"Te voy a pasar un texto, haz un analisis de sentimiento y responde simplemente con 'positivo' ,'negativo' o 'nuetral'. Aca esta el texto : {texto}"}])
            
            sentimiento = completion.choices[0].message.content.lower()



            #Fila que sera añadida a la base de datos 
            fila_articulo = {"Query":query,  "Titulo":titulo,"Fecha":fecha, "Url":url_final,"Texto":texto , "Sentimiento": sentimiento}

            df.loc[len(df)] = fila_articulo

        else:
            print("Error con la extracción del artículo.")
    
    #Excepciones    
    except TimeoutException:
        print(f"No se encontró artículo en la ruta {article_xpath}.")
    
    except Exception as e:
        print(f"Error al procesar artículo: {e}")


for key in queries:

    print(f"Inicio de temas de {key}\n")
    
    for q in queries[key]:
        print(f"Inicio de sub-busqueda de {q}\n")
        
        url_google_news = f"https://news.google.com/search?q=%22{q}%22%20when%3A{cantDias}d&hl=es-419&gl=AR&ceid=AR%3Aes-419"
        
        with webdriver.Chrome() as driver:
            driver.get(url_google_news)
            time.sleep(8)

            #identificadores de los articulos a tomar, por default se toman los primeros 7
            xpaths = [
                '//*[@id="yDmH0d"]/c-wiz/div/main/div[2]/c-wiz/c-wiz[1]/c-wiz/article/div[1]/div[1]/a',
                '//*[@id="yDmH0d"]/c-wiz/div/main/div[2]/c-wiz/c-wiz[2]/c-wiz/article/div[1]/div[1]/a',
                '//*[@id="yDmH0d"]/c-wiz/div/main/div[2]/c-wiz/c-wiz[3]/c-wiz/article/div[1]/div[1]/a',
                '//*[@id="yDmH0d"]/c-wiz/div/main/div[2]/c-wiz/c-wiz[4]/c-wiz/article/div[1]/div[1]/a',
                '//*[@id="yDmH0d"]/c-wiz/div/main/div[2]/c-wiz/c-wiz[5]/c-wiz/article/div[1]/div[1]/a',
                '//*[@id="yDmH0d"]/c-wiz/div/main/div[2]/c-wiz/c-wiz[6]/c-wiz/article/div[1]/div[1]/a',
                '//*[@id="yDmH0d"]/c-wiz/div/main/div[2]/c-wiz/c-wiz[7]/c-wiz/article/div[1]/div[1]/a'
            ]

            for xpath in xpaths:
                process_article(driver, xpath,q)
        
        print(f"Fin de sub-busqueda de {q}\n")
    print(f"Fin de busqueda de {key}\n")

Inicio de temas de Politica

Inicio de sub-busqueda de Milei

Fin de sub-busqueda de Milei

Inicio de sub-busqueda de Caputo

Fin de sub-busqueda de Caputo

Fin de busqueda de Politica

Inicio de temas de Economia

Inicio de sub-busqueda de Comercio

Fin de sub-busqueda de Comercio

Inicio de sub-busqueda de Finanzas

Fin de sub-busqueda de Finanzas

Inicio de sub-busqueda de Pyme

Fin de sub-busqueda de Pyme

Fin de busqueda de Economia

Inicio de temas de Actual

Inicio de sub-busqueda de RIGI

No se encontró artículo en la ruta //*[@id="yDmH0d"]/c-wiz/div/main/div[2]/c-wiz/c-wiz[4]/c-wiz/article/div[1]/div[1]/a.
No se encontró artículo en la ruta //*[@id="yDmH0d"]/c-wiz/div/main/div[2]/c-wiz/c-wiz[5]/c-wiz/article/div[1]/div[1]/a.
No se encontró artículo en la ruta //*[@id="yDmH0d"]/c-wiz/div/main/div[2]/c-wiz/c-wiz[6]/c-wiz/article/div[1]/div[1]/a.
No se encontró artículo en la ruta //*[@id="yDmH0d"]/c-wiz/div/main/div[2]/c-wiz/c-wiz[7]/c-wiz/article/div[1]/div[1]/a.
Fin de sub-bus

In [26]:
#Pasamos nuestros query's de un formato de url a uno coloquial
for index, row in df.iterrows():
    df.at[index, 'Query'] = row['Query'].replace('%20', ' ')

current_time = time.localtime()

# Format the time into a string with the format "day_month"
dia_mes = time.strftime("%d_%m", current_time)

# Especificar el nombre del archivo de Excel
nombre_archivo = f"baseDeDatos__noticias{dia_mes}.xlsx"

# Guardar el DataFrame en un archivo de Excel
df.to_excel(nombre_archivo, index=False)

print(f"El DataFrame se ha guardado en el archivo {nombre_archivo}")

El DataFrame se ha guardado en el archivo baseDeDatos__noticias05_08.xlsx


In [25]:
df


,Query,Titulo,Fecha,Url,Texto,Sentimiento
0,Milei,"Javier Milei, en vivo: las últimas medidas del...",05-08-2024,https://www.lanacion.com.ar/politica/javier-mi...,Escuchar\n\n📌08.18 | La reacción de Sturzenegg...,positivo
1,Milei,“En la Argentina del Presidente Milei las empr...,30-07-2024,https://www.argentina.gob.ar/noticias/en-la-ar...,\n\n\n\nEl Vocero Presidencial Manuel Adorni a...,neutral
2,Milei,"Los viajes de Javier Milei al exterior, uno po...",13-07-2024,https://chequeado.com/el-explicador/uno-por-un...,"Estados Unidos fue el destino más frecuente, c...",neutral
3,Milei,Kicillof: “La única realidad es que Milei deci...,NaN,https://www.gba.gob.ar/comunicacion_publica/ga...,"“Estamos ante un hecho de enorme gravedad, una...",negativo
4,Milei,Milei tiene que dar las gracias a Kicillof | E...,NaN,https://www.pagina12.com.ar/757570-milei-tiene...,Sin la presentación pública de un informe técn...,negativo
5,Milei,"Milei desautoriza a su vicepresidenta, Victori...",19-07-2024,https://elpais.com/argentina/2024-07-19/milei-...,La defensa que hizo la vicepresidenta Victoria...,negativo
6,Milei,Cruzan a Milei por vincular a la izquierda con...,15-07-2024,https://www.clacso.org/cruzan-a-milei-por-vinc...,Compartimos una nota publicada en Página 12 el...,positivo
7,Caputo,Luis Caputo se reunió con Agentes de Liquidaci...,30-07-2024,https://www.argentina.gob.ar/noticias/luis-cap...,"El ministro de Economía, Luis Caputo, mantuvo ...",neutral
8,Caputo,Sáenz y Caputo avanzaron en un acuerdo para qu...,NaN,https://www.salta.gob.ar/prensa/noticias/saenz...,El gobernador de Salta Gustavo Sáenz y el mini...,positivo
9,Caputo,"El plan Caputo de ""secar la plaza (y los bolsi...",NaN,https://www.pagina12.com.ar/757653-el-plan-cap...,El impacto de la recesión sigue sintiéndose co...,negativo
